Edge letter recognition prototype model
===

In [ ]:
import cv2
import numpy as np

from shredsim import classifier
from shredsim import utils
from shredsim import dataset
from shredsim import border as border_m

import matplotlib
matplotlib.rc('font', family='Arial')
%pylab inline

In [ ]:
source, shred_config = utils.load_targets()
mask = shred_config.mask
imshow(source, cmap='gray')
print "Document used for training/testing."

In [ ]:
graph = utils.cut_to_shreds(source.shape, shred_config)
print 'Total shreds:',len(graph)

for node in graph.nodes():
    if not utils.is_good_node(source[graph.node[node]['slice']]):
        graph.remove_node(node)
print "'Good', non-empty shreds:", len(graph)

In [ ]:
pattern = np.zeros(source.shape)
for n in graph:
    pattern[dataset.to_slice(n, shred_config.mask.shape)] = shred_config.mask
imshow(pattern, cmap='gray')
print "Non-empty shreds layout."

In [ ]:
# Pick random shred for further processing.
import random
key1 = random.choice(graph.nodes())
# Apply shred mask to a random rectangular doc slice.
subplot(131)
title("Mask")
imshow(mask, cmap='gray')
subplot(132)
title("Doc region")
imshow(source[graph.node[key1]['slice']], cmap='gray')
subplot(133)
title("Masked region")
imshow(utils.masked_shred(source[graph.node[key1]['slice']], mask), cmap='gray')
print mask.shape

In [ ]:
# Pad shred mask and determine border points.
# A window slides over border points to find partial characters.

padded_mask = dataset.pad_image(mask, dataset.WINDOW_SIDE)

border_obj = border_m.ShredMaskBorder(padded_mask)
border = border_obj.get_border_mask()
figure(num=None, figsize=(5, 10), facecolor='w', edgecolor='k')
subplot(131)
title("Padded mask")
imshow(padded_mask, cmap='gray')
subplot(132)
title("Inner border")
imshow(border, cmap='gray')
subplot(133)
title("Inner border position")
imshow(cv2.bitwise_xor(padded_mask, border), cmap='gray')

border_points = border_obj.get_border_points()  # Coordinates within padded mask/shred image.
print padded_mask.shape

In [ ]:
all_samples, all_labels = classifier.get_dataset()

In [ ]:
cl = classifier.get_classifier()
cl2 = classifier.DBNNNClassifier(cl, (all_samples, all_labels))

In [ ]:
shred = utils.masked_shred(source[graph.node[key1]['slice']], mask)

padded_shred = dataset.pad_image(shred, dataset.WINDOW_SIDE)
imshow(shred, cmap='gray')

# Windows from this shred's borders.
samples = []
sample_coords = []
c = 0
for b in border_points:   
    window_origin = b - dataset.WINDOW_SHAPE / 2
    window_slice_index = dataset.to_slice(window_origin, dataset.WINDOW_SHAPE)
    window = padded_shred[window_slice_index]

    mask_window = padded_mask[window_slice_index]
    outermost = utils.preserve_outermost(window, mask_window)
    
    if not utils.is_good_node(outermost):
        continue
    
    sample = (outermost / 255.).flatten()
    samples.append(sample)
    sample_coords.append(window_origin)
    
    c += 1

    #output = padded_shred.copy()
    #cv2.imshow("Window", window)
    #cv2.rectangle(output, tuple((b-window_shape/2)[::-1]), tuple((b + window_shape/2)[::-1]), (255,0,0,255))
    #cv2.imshow("Shred", output)
    #cv2.waitKey(10)
samples = np.array(samples)
print samples.shape

In [ ]:
res = cl.predict(samples)

In [ ]:
#res2 = cl2.predict(samples)

In [ ]:
import collections

all_labels = set(res)
num_labels = len(all_labels)
colors = np.random.randint(0, 256, size=(num_labels, 4))
colors[:, 3] = 255

label2color = dict(zip(all_labels, colors))
print "Total labels:", num_labels

top_labels_to_pick = 7
top_labels = [x[0] for x in collections.Counter(res).most_common(top_labels_to_pick)]

row_height = 80
legend = np.ones(((top_labels_to_pick+1) * row_height, 400, 3), dtype=shred.dtype) * 255

output = cv2.cvtColor(padded_shred.copy(), cv2.cv.CV_GRAY2BGRA)
for coords, label in zip(sample_coords, res):
    if label not in top_labels: continue
    coords = coords +  dataset.WINDOW_SHAPE / 2
    cv2.circle(output, (coords[1], coords[0]), 5, label2color[label], thickness=-1)

    
covered_area = cv2.dilate(border, np.ones(dataset.WINDOW_SHAPE/2))
        
figure(num=None, figsize=(15, 10))
subplot(171)
imshow(dataset.pad_image(source[graph.node[key1]['slice']], dataset.WINDOW_SIDE), cmap='gray')
subplot(172)
imshow(padded_mask, cmap='gray')
subplot(173)
imshow(padded_shred, cmap='gray')
subplot(174)
imshow(covered_area, cmap='gray')
subplot(175)
imshow(cv2.bitwise_and(padded_shred, padded_shred, mask=covered_area), cmap='gray')
subplot(176)
imshow(output)
subplot(177)
    
for i, label in enumerate(top_labels):
    cv2.circle(legend, (30, (i+1) * row_height), 15, label2color[label][:3], thickness=-1)
    text(60, (i+1.1) * row_height, unicode("%s (%d)" % (label, collections.Counter(res)[label]), 'utf-8'), color='black', fontsize=20)

imshow(legend)


In [ ]:
print len(res)

In [ ]:
len(all_samples)

In [ ]:
i = np.random.randint(0, len(res))

print i, len(res)

image = (samples[i] * 255).reshape(dataset.WINDOW_SHAPE).astype("uint8")

output = padded_shred.copy()
point_a = sample_coords[i] [::-1]
point_b = tuple(point_a + dataset.WINDOW_SHAPE)
cv2.rectangle(output, tuple(point_a), point_b, (250), 2)

subplot(141)
imshow(image, cmap='gray')
subplot(142)
imshow(output, cmap='gray')

proba = cl.predict_proba(np.array([samples[i]]))[0]
proba = (proba*100).astype('int8')

uber_prediction = cl2.predict(np.array([samples[i]]))

subplot(122)
plt.bar(range(len(proba)), proba, align='center')
_ = plt.xticks(range(len(proba)), map(lambda x: unicode(x, 'utf-8'), cl.classes_), rotation=25)

print "Sample window classification"
for c, prob in zip(cl.classes_[proba>0], proba[proba>0]):
    print c, prob

print "Custom super classifier prediction:", uber_prediction[0]

In [ ]:
flat_image = image.flatten().astype(np.float32)
all_distances = [np.linalg.norm(flat_image - sample) for sample in all_samples]

In [ ]:
closest_idx = np.argmin(all_distances)
closest = all_samples[closest_idx].reshape(dataset.WINDOW_SHAPE)
closest_label = all_labels[closest_idx]

subplot(121)
title("Window image")
imshow(image, 'gray')
subplot(122)
title("Closest match from the dataset")
imshow(closest, 'gray')

print "Bitwise closest image"
print "Closest label:", closest_label, "distance:", all_distances[closest_idx], "idx:", closest_idx